In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.graph_objects as go
import re
import warnings
warnings.filterwarnings("ignore")

#Silahkan Tambahkan Package lainnya jika dibutuhkan

In [2]:
from google.colab import drive

# Replace 'SHARED_FOLDER_LINK' with the actual link of the shared folder
folder_link = 'https://drive.google.com/drive/folders/1Cd1iNDv4gfTUX41myo2m4Fazz_zA9HBG?usp=drive_link'

# Extract the folder ID from the link (you can find it in the URL after 'folders/')
folder_id = folder_link.split('/')[-1]

# Mount the shared folder in Colab
drive.mount('/content/drive', force_remount=True)

# Define the path to the shared folder without including the link
folder_path = '/content/drive/Shared with me/' + folder_id + '/'

Mounted at /content/drive


In [4]:
df = pd.read_csv('Training Data.csv')

## Membaca Dataset Loan Predictor Based On Customer Behavior


In [5]:
df.head()

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


Mengubah kolom Id menjadi kolom index

In [6]:
df.set_index('Id', inplace=True)
df.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


## Descriptive Statistik Analysis

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252000 entries, 1 to 252000
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Income             252000 non-null  int64 
 1   Age                252000 non-null  int64 
 2   Experience         252000 non-null  int64 
 3   Married/Single     252000 non-null  object
 4   House_Ownership    252000 non-null  object
 5   Car_Ownership      252000 non-null  object
 6   Profession         252000 non-null  object
 7   CITY               252000 non-null  object
 8   STATE              252000 non-null  object
 9   CURRENT_JOB_YRS    252000 non-null  int64 
 10  CURRENT_HOUSE_YRS  252000 non-null  int64 
 11  Risk_Flag          252000 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 25.0+ MB


Dari informasi di atas, dapat diketahui bahwa nama kolom dengan tipe datanya sudah sesuai.

In [8]:
# Mengetahui jumlah kolom dan baris
jumlah_baris, jumlah_kolom = df.shape

# Menampilkan hasil
print("Jumlah Baris:", jumlah_baris)
print("Jumlah Kolom:", jumlah_kolom)

Jumlah Baris: 252000
Jumlah Kolom: 12


Dari informasi di atas, dapat diketahui bahwa pada dataset ini nama kolom beserta tipe datanya sudah sesuai. Dengan jumlah baris sebanyak 252.000 dan jumlah kolom sebanyak 13.

In [9]:
df.isnull().sum()

Income               0
Age                  0
Experience           0
Married/Single       0
House_Ownership      0
Car_Ownership        0
Profession           0
CITY                 0
STATE                0
CURRENT_JOB_YRS      0
CURRENT_HOUSE_YRS    0
Risk_Flag            0
dtype: int64

Dari informasi di atas, diketahui bahwa tidak ada nilai kosong pada masing-masing kolom.

In [10]:
df.duplicated().sum()

208810

Dari informasi di atas, diketahui bahwa tidak ada nilai duplikat pada masing-masing kolom.

In [ ]:
# Mengganti nama kolom
df.rename(columns={'Id': 'id', 'Income': 'income', 'Age': 'age', 'Experience': 'experience', 'Married/Single': 'marital_status', 'House_Ownership': 'house_ownership', 'Car_Ownership': 'car_ownership', 'Profession': 'profession', 'CITY': 'city', 'STATE': 'state', 'CURRENT_JOB_YRS': 'current_job_yrs', 'CURRENT_HOUSE_YRS': 'current_house_yrs', 'Risk_Flag': 'risk_flag'}, inplace=True)

# Menampilkan DataFrame setelah perubahan
print("\nDataFrame Setelah Perubahan Nama Kolom:")
df.info()

Nama kolom sudah seragam.

In [ ]:
nums = ['id', 'income', 'age', 'experience', 'current_job_yrs', 'current_house_yrs', 'risk_flag']
cats = ['marital_status', 'house_ownership', 'car_ownership', 'profession', 'city', 'state']

# melihat deskripsi kolom kategorikal & nilai unique
df.select_dtypes(include='object').nunique()

In [ ]:
# cek nilai unique di kolom kategorikal
for col in df.select_dtypes(include=['object']).columns:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# Menghilangkan angka dan tanda kurung siku dari data di kolom "city"
df.city = df['city'].str.replace(r'\[\d+\]', '')
df.city = df.city.str.replace('_', ' ')

# Menampilkan DataFrame setelah perubahan
print(df.city)

In [ ]:
# Menampilkan nilai unik dan frekuensinya pada kolom 'city' yang sudah diurutkan dari frekuensi terkecil
sorted_value_counts_city1 = df.city.value_counts().sort_index()
print("Nilai Unik dan Frekuensi yang Sudah Diurutkan pada Kolom 'city':")
print(sorted_value_counts_city1)

# Menampilkan jumlah nilai unik
jumlah_nilai_unik_city1 = df.city.nunique()
print("\nJumlah Nilai Unik pada Kolom 'city':", jumlah_nilai_unik_city1)

In [ ]:
# Menghilangkan angka dan tanda kurung siku dari data di kolom "state"
df.state = df['state'].str.replace(r'\[\d+\]', '')
df.state = df.state.str.replace('_', ' ')

# Menampilkan DataFrame setelah perubahan
print(df.state)

In [ ]:
# Menampilkan nilai unik dan frekuensinya pada kolom 'state' yang sudah diurutkan dari frekuensi terkecil
sorted_value_counts_state1 = df.state.value_counts().sort_index()
print("Nilai Unik dan Frekuensi yang Sudah Diurutkan pada Kolom 'state':")
print(sorted_value_counts_state1)

# Menampilkan jumlah nilai unik
jumlah_nilai_unik_state1 = df.state.nunique()
print("\nJumlah Nilai Unik pada Kolom 'state':", jumlah_nilai_unik_state1)

In [ ]:
# melihat data summary
df[nums].describe()

In [ ]:
# melihat data summary
df[cats].describe()

**Kesimpulan dari Descriptive Statistics**

1. Data terdiri dari 252000 baris dan 13 kolom serta tidak ada nilai null.
2. 5 kolom dengan variabel kontinu : income , age , experience , current_job_yrs , current_house_yrs.
3. 8 kolom dengan variabel kategori : id , married_single,house_ownership, car_ownership , profession , city , state , risk_flag (target variabel).

## Univariate Analysis

**Distribusi Histogram**

In [ ]:
df.hist( figsize = (10, 8) )
plt.show()

**Skewness**

In [ ]:
# Visualize skewness using seaborn
plt.figure(figsize=(12, 6))

for i, column in enumerate(nums, 1):
    plt.subplot(2, 4, i)
    sns.histplot(df[column], kde=True)
    plt.title(f'Skewness: {skew(df[column]):.2f}')

plt.tight_layout()
plt.show()

Distribusi data pada dataset ini adalah multimodal.

**Outlier**

In [ ]:
plt.figure(figsize=(10, 5))
for i, column in enumerate(nums, 1):
    plt.subplot(1, len(nums), i)
    sns.boxplot(y=df[column], color='yellow', orient='v')
    plt.title(f'{column}')

plt.tight_layout()
plt.show()

Dari informasi di atas, dapat diketahui bahwa pada dataset ini tidak ada outlier.

**Distribusi Count per Kolom**

In [ ]:
# Membuat satu subplot untuk setiap count plot
plt.figure(figsize=(20, 15))

for i, column in enumerate(cats, 1):
    plt.subplot(2, 3, i)
    sns.countplot(data=df, y=column, palette="nipy_spectral")
    plt.title(f'{column}')

plt.tight_layout()
plt.show()

**Kesimpulan dari Distribusi Data**

1. Kolom-kolom seperti city, state, dan profession memiliki data yang sangat banyak, sehingga diperlukan pengelompokan data untuk menggabungkan kategori serupa menjadi satu kategori baru. Tujuannya adalah untuk menyederhanakan data.
2. Ditemukan ketidakseimbangan dominasi kategori pada kolom married/single dan house_ownership, yang dapat memengaruhi hasil analisis. Oleh karena itu, perlu dipertimbangkan apakah tindakan seperti oversampling atau undersampling diperlukan untuk menangani ketidakseimbangan tersebut.

**Pengelompokan Kategori Data Numerik Berdasarkan Kuartil**



> Kategori Kolom income



In [ ]:
df_income = pd.DataFrame(df)

# Menghitung kuartil
quartiles = df_income['income'].quantile([0, 0.33, 0.67, 1])

# Membuat fungsi untuk mengkategori income
def categorize_income(income):
    if income <= quartiles[0.33]:
        return 'low'
    elif income <= quartiles[0.67]:
        return 'middle'
    else:
        return 'high'

# Menerapkan fungsi ke kolom baru 'Income_Category'
df_income['income_category'] = df_income['income'].apply(categorize_income)

In [ ]:
# Mengurutkan nilai pada kategori 'low'
low_data_sorted = df_income[df_income['income_category'] == 'low'].sort_values(by='income')

# Menampilkan data yang sudah diurutkan
low_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'low'
low_data_sorted = df_income[df_income['income_category'] == 'low'].sort_values(by='income', ascending=False)

# Menampilkan data yang sudah diurutkan
low_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'middle'
middle_data_sorted = df_income[df_income['income_category'] == 'middle'].sort_values(by='income')

# Menampilkan data yang sudah diurutkan
middle_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'middle'
middle_data_sorted = df_income[df_income['income_category'] == 'middle'].sort_values(by='income', ascending=False)

# Menampilkan data yang sudah diurutkan
middle_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'high'
high_data_sorted = df_income[df_income['income_category'] == 'high'].sort_values(by='income')

# Menampilkan data yang sudah diurutkan
high_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'high'
high_data_sorted = df_income[df_income['income_category'] == 'high'].sort_values(by='income', ascending=False)

# Menampilkan data yang sudah diurutkan
high_data_sorted.head()



> Kategori Kolom age



In [ ]:
df_age = pd.DataFrame(df)

def classify_age(age):
    if age < 30:
        return 'young'
    elif age < 60:
        return 'adult'
    else:
        return 'elder'


# Menerapkan fungsi ke kolom baru 'Age_Category'
df_age['age_category'] = df_age['age'].apply(classify_age)

In [ ]:
# Mengurutkan nilai pada kategori 'young'
young_data_sorted = df_age[df_age['age_category'] == 'young'].sort_values(by='age')

# Menampilkan data yang sudah diurutkan
young_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'young'
remaja_data_sorted = df_age[df_age['age_category'] == 'young'].sort_values(by='age', ascending=False)
# Menampilkan data yang sudah diurutkan
remaja_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'adult'
adult_data_sorted = df_age[df_age['age_category'] == 'adult'].sort_values(by='age')

# Menampilkan data yang sudah diurutkan
adult_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'adult'
adult_data_sorted = df_age[df_age['age_category'] == 'adult'].sort_values(by='age', ascending=False)

# Menampilkan data yang sudah diurutkan
adult_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'elder'
elder_data_sorted = df_age[df_age['age_category'] == 'elder'].sort_values(by='age')

# Menampilkan data yang sudah diurutkan
elder_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'elder'
elder_data_sorted = df_age[df_age['age_category'] == 'elder'].sort_values(by='age', ascending=False)

# Menampilkan data yang sudah diurutkan
elder_data_sorted.head()



> Kategori Kolom experience



In [ ]:
df_experience = pd.DataFrame(df)

# Menghitung kuartil
quartiles = df_experience['experience'].quantile([0, 0.33, 0.67, 1])

# Membuat fungsi untuk mengkategori experience
def categorize_experience(experience):
    if experience <= quartiles[0.33]:
        return 'beginner'
    elif experience <= quartiles[0.67]:
        return 'advance'
    else:
        return 'expert'

# Menerapkan fungsi ke kolom baru 'Experience_Category'
df_experience['experience_category'] = df_experience['experience'].apply(categorize_experience)

In [ ]:
# Mengurutkan nilai pada kategori 'beginner'
experience_data_sorted = df_experience[df_experience['experience_category'] == 'beginner'].sort_values(by='experience')

# Menampilkan data yang sudah diurutkan
experience_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'beginner'
experience_data_sorted = df_experience[df_experience['experience_category'] == 'beginner'].sort_values(by='experience', ascending=False)

# Menampilkan data yang sudah diurutkan
experience_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'advance'
experience_data_sorted = df_experience[df_experience['experience_category'] == 'advance'].sort_values(by='experience')

# Menampilkan data yang sudah diurutkan
experience_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'advance'
experience_data_sorted = df_experience[df_experience['experience_category'] == 'advance'].sort_values(by='experience', ascending=False)

# Menampilkan data yang sudah diurutkan
experience_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'expert'
experience_data_sorted = df_experience[df_experience['experience_category'] == 'expert'].sort_values(by='experience')

# Menampilkan data yang sudah diurutkan
experience_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'expert'
experience_data_sorted = df_experience[df_experience['experience_category'] == 'expert'].sort_values(by='experience', ascending=False)

# Menampilkan data yang sudah diurutkan
experience_data_sorted.head()



> Kategori Kolom current_job_yrs



In [ ]:
df_job = pd.DataFrame(df)

# Menghitung kuartil
quartiles = df_job['current_job_yrs'].quantile([0, 0.33, 0.67, 1])

# Membuat fungsi untuk mengkategori job
def categorize_job(job):
    if job <= quartiles[0.33]:
        return 'junior'
    elif job <= quartiles[0.67]:
        return 'senior'
    else:
        return 'mid_level'

# Menerapkan fungsi ke kolom baru 'job_category'
df['job_category'] = df['current_job_yrs'].apply(categorize_job)

In [ ]:
# Mengurutkan nilai pada kategori 'junior'
job_data_sorted = df_job[df_job['job_category'] == 'junior'].sort_values(by='current_job_yrs')

# Menampilkan data yang sudah diurutkan
job_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'junior'
job_data_sorted = df_job[df_job['job_category'] == 'junior'].sort_values(by='current_job_yrs', ascending=False)

# Menampilkan data yang sudah diurutkan
job_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'senior'
job_data_sorted = df_job[df_job['job_category'] == 'senior'].sort_values(by='current_job_yrs')

# Menampilkan data yang sudah diurutkan
job_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'senior'
job_data_sorted = df_job[df_job['job_category'] == 'senior'].sort_values(by='current_job_yrs', ascending=False)
# Menampilkan data yang sudah diurutkan
job_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'mid_level'
job_data_sorted = df_job[df_job['job_category'] == 'mid_level'].sort_values(by='current_job_yrs')
# Menampilkan data yang sudah diurutkan
job_data_sorted.head()

In [ ]:
# Mengurutkan nilai pada kategori 'mid_level'
job_data_sorted = df_job[df_job['job_category'] == 'mid_level'].sort_values(by='current_job_yrs', ascending=False)
# Menampilkan data yang sudah diurutkan
job_data_sorted.head()



> Merged Dataframe



In [ ]:
# Menggabungkan df_income dan df_job berdasarkan indeks
merged_df = df_income[['income_category']].join(df_age[['age_category']])

# Menggabungkan dengan df_age berdasarkan indeks
merged_df = merged_df.join(df_experience[['experience_category']])

# Menggabungkan dengan df_experience berdasarkan indeks
merged_df = merged_df.join(df_job[['job_category']])


# Menampilkan hasil
merged_df

In [ ]:
nums_group = ['income_category', 'age_category', 'experience_category', 'job_category']

In [ ]:
# Mengatur warna dan ukuran
sns.set(style="darkgrid")
plt.figure(figsize=(11, 9))

# Loop untuk membuat subplot
for i in range(0, len(nums_group)):
    plt.subplot(3, 2, i+1)
    sns.countplot(x=merged_df[nums_group[i]], palette='inferno')
    plt.title(f'Distribusi {nums_group[i]}')
    plt.tight_layout()

# Menampilkan hasil
plt.show()

**Pengelompokkan Data Categorical**

In [ ]:
#Data professions
profession_group = {
    'engineer tech': ['Computer_hardware_engineer', 'Industrial_Engineer', 'Mechanical_engineer', 'Chemical_engineer',
                        'Biomedical_Engineer', 'Software_Developer', 'Petroleum_Engineer', 'Civil_engineer',
                        'Design_Engineer', 'Engineer','Technician', 'Technology_specialist', 'Computer_operator','Technical_writer'],
    'design art': ['Web_designer', 'Fashion_Designer', 'Graphic_Designer', 'Artist', 'Designer','Comedian','Chef'],
    'health': ['Dentist', 'Surgeon', 'Microbiologist', 'Nurse','Psychologist','Physician'],
    'consultant': ['Financial_Analyst', 'Consultant', 'Chartered_Accountant','Lawyer', 'Economist','Surveyor', 'Analyst',
                    'Architect','Statistician', 'Scientist', 'Geologist'],
    'office management': ['Magistrate', 'Secretary', 'Official', 'Hotel_Manager','Drafter','Librarian'],
    'aviation': ['Flight_attendant', 'Aviator','Air_traffic_controller'],
    'gov': ['Police_officer','Politician','Army_officer', 'Firefighter', 'Civil_servant']
}

In [ ]:
df['profession_group'] = df['profession'].apply(lambda x: next((key for key, value in profession_group.items() if x in value), 'other'))

In [ ]:
def city_group(city):
    if city in ['Delhi city', 'Mumbai', 'Kolkata', 'Chennai', 'Bangalore']:
        return 'Metro'
    elif city in ['Ahmedabad', 'Hyderabad', 'Pune', 'Surat', 'Jaipur', 'Lucknow', 'Kanpur', 'Nagpur', 'Visakhapatnam', 'Indore', 'Thane',
                  'Bhopal', 'Pimpri-Chinchwad', 'Patna', 'Vadodara', 'Ghaziabad', 'Ludhiana', 'Agra', 'Nashik', 'Faridabad', 'Meerut', 'Rajkot',
                  'Varanasi', 'Srinagar', 'Amritsar', 'Allahabad', 'Jabalpur', 'Gwalior', 'Vijayawada', 'Jodhpur', 'Raipur', 'Kota', 'Guwahati', 'Chandigarh city']:
        return 'Urban'
    elif city in ['Navi Mumbai', 'Kalyan-Dombivli', 'Vasai-Virar', 'Mira-Bhayandar', 'Thiruvananthapuram', 'Bhiwandi', 'Noida', 'Bhopal', 'Howrah',
                  'Saharanpur', 'Berhampur', 'Suryapet', 'Muzaffarpur', 'Nadiad', 'Siliguri', 'Bhavnagar', 'Kurnool', 'Tenali', 'Satna', 'Nandyal',
                  'Etawah', 'Morena', 'Ballia', 'Machilipatnam', 'Mau', 'Machilipatnam', 'Bhagalpur', 'Siwan', 'Meerut', 'Dibrugarh', 'Gaya', 'Darbhanga',
                  'Hajipur', 'Mirzapur', 'Akola', 'Satna', 'Motihari', 'Jalna', 'Ramgarh', 'Ozhukarai', 'Saharsa', 'Munger', 'Farrukhabad', 'Nangloi Jat',
                  'Thoothukudi', 'Nagercoil', 'Rourkela', 'Jhansi', 'Sultan Pur Majra']:
        return 'Suburban'
    else:
        return 'Rural'

In [ ]:
df['city_group'] = df['city'].apply(city_group)

In [ ]:
state_group = {
    'north_zone': ['Uttar Pradesh', 'Haryana', 'Jammu and Kashmir', 'Punjab', 'Uttarakhand', 'Chandigarh', 'Delhi', 'Himachal Pradesh'],
    'east_zone': ['Bihar', 'Jharkhand', 'Odisha', 'West Bengal', 'Assam', 'Sikkim', 'Tripura', 'Mizoram', 'Manipur'],
    'south_zone': ['Andhra Pradesh', 'Tamil Nadu', 'Karnataka', 'Telangana', 'Kerala', 'Puducherry'],
    'west_zone': ['Maharashtra', 'Gujarat', 'Rajasthan', 'Chhattisgarh', 'Madhya Pradesh'],
}

In [ ]:
df['state_group'] = df['state'].apply(lambda x: next((key for key, value in state_group.items() if x in value), 'Other'))

In [ ]:
df

In [ ]:
cats_group = ['profession_group', 'city_group', 'state_group']

In [ ]:
# Mengatur warna dan ukuran
sns.set(style="darkgrid")
plt.figure(figsize=(20, 18))

# Loop untuk membuat subplot
for i in range(0, len(cats_group)):
    plt.subplot(3, 2, i+1)
    sns.countplot(x=df[cats_group[i]], palette='inferno')
    plt.xticks(rotation=90)
    plt.title(f'Distribusi {cats_group[i]}')
    plt.tight_layout()

# Menampilkan hasil
plt.show()

Setelah dilakukan grouping pada kolom-kolom yang memiliki kategori yang luas, countplot lebih mudah dibaca dan selanjutnya akan diproses lebih lanjut pada Multivariate Analysis.

## Multivariate Analysis

## Business Insight and Recomendation